In [1]:
import time
ini=time.time()

import datetime

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier as RFC_
from sklearn.ensemble import RandomForestRegressor as RFR_
from sklearn import metrics

comex_ncm=c=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv')

# Filtra top
top_nick=c.groupby('nick').KG_LIQUIDO.sum().sort_values().index[-2:]
top_uf=c.groupby('SG_UF_NCM').KG_LIQUIDO.sum().sort_values().index[-2:]
top_pais=c.groupby('CO_PAIS').KG_LIQUIDO.sum().sort_values().index[-2:]
top_urf=c.groupby('CO_URF').KG_LIQUIDO.sum().sort_values().index[-2:]
c=c[(c.nick.isin(top_nick))&(c.SG_UF_NCM.isin(top_uf))&(c.CO_PAIS.isin(top_pais))&(c.CO_URF.isin(top_urf))]

In [2]:
md_c=15
md_r=1
l=20 # anos
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
%run 'func_prev_comex_ncm.ipynb'

preve_comex_ncm(comex_ncm, target_cols=['oc','KG_LIQUIDO'], cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF'],md_c=None,md_r=1,comp_min_anos=5)


In [3]:
q=pd.DataFrame()
for md_c in [15,25]:
    for md_r in [1,5]:
        comex_ncm_t=comex_ncm.copy()
        while (comex_ncm_t.date.max()-pd.DateOffset(years=l))>=comex_ncm_t.date.min():
            
            comex_ncm_t2=comex_ncm_t[comex_ncm_t.date>(comex_ncm_t.date.max()-pd.DateOffset(years=l))]
            
            test_pred=preve_comex_ncm(comex_ncm_t2,
                                      target_cols=['oc','KG_LIQUIDO'],
                                      cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF'],
                                      md_c=md_c,
                                      md_r=md_r,
                                      comp_min_anos=5)
            
            if len(test_pred)>0:
                test_pred=test_pred.assign(ano='max')
                
                test_pred.loc[test_pred.CO_ANO==test_pred.CO_ANO.min(),'ano']='min'
                
                test_pred=test_pred.assign(md_c=md_c).assign(md_r=md_r)
                
                q=pd.concat([q,test_pred]).drop_duplicates()
                
                comex_ncm_t=comex_ncm_t[comex_ncm_t.date<comex_ncm_t.date.max()]

In [4]:
q.to_csv('q.csv',index=False)

In [16]:
fim=time.time()
print('Duração:',round((fim-ini)/60,1),' minutos.')

Duração: 2.6  minutos.
